# Twitch Insight: Data Collector & Collaboration Network Analyzer (v3 - Enhanced Prints)

This notebook automates the process of collecting data from the Twitch API, store it in a local SQLite database, and perform analysis, with a special focus on understanding collaboration networks between streamers. It identifies potential collaborations by detecting `@mentions` in video titles and descriptions, and also explores community structures within these networks. The primary interface for data collection, processing, and exploration is a Jupyter Notebook.

**Key Features:**
- Fetches top streams/categories periodically.
- Fetches channel details and video archives.
- Processes video titles/descriptions for `@mentions`.
- Looks up mentioned users via API if not in the database.
- Stores collaboration data (frequency, duration, recency).
- Processes mentions atomically per-video using DB transactions.
- Includes a refresh cycle for updating random channels.
- Provides data exploration for channels, videos, and the collaboration network.

**Modules Used:**
- `config.py`: Configuration (API keys, constants). Requires `.env` file.
- `database.py`: SQLite database interactions (schema, CRUD, upserts).
- `twitch_api.py`: Twitch Helix API communication (auth, rate limits).
- `network_utils.py`: Mention extraction and validation.

In [ ]:
# Cell 1: Imports and Setup
import logging
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, timezone
import re  # For parsing duration
import math  # For log scale checks
import random  # For refresh cycle
import networkx as nx  # For graph analysis
from ipywidgets import Text, Button, Dropdown, Output, VBox, Layout  # For interactive viz
from IPython.display import display, HTML  # For displaying widgets and potentially HTML

# --- Local Notebook Configuration ---
# Set to True for detailed progress and time estimations, False for minimal output.
VERBOSE_MODE = True

# Import custom modules
try:
    import config
    import database
    import twitch_api
    import network_utils
except ImportError as e:
    print(f"Error importing custom modules: {e}")
    print("Please ensure config.py, database.py, twitch_api.py, and network_utils.py are in the same directory.")
    raise SystemExit("Stopping notebook due to missing modules.")

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')

# --- Print Configuration ---
config.print_config()  # Call the function from config.py
print(f"\n[Notebook Settings]\n  VERBOSE_MODE: {VERBOSE_MODE}")

# --- Initialize ---
db_conn = None
api_client = None
try:
    print("\nInitializing database connection...")
    # Use settings from config.py
    db_conn = database.get_db_connection(config.DATABASE_NAME)
    database.initialize_database(db_conn)
    print(f"Database '{config.DATABASE_NAME}' initialized.")

    print("Initializing Twitch API client...")
    api_client = twitch_api.TwitchAPIClient(
        client_id=config.TWITCH_CLIENT_ID,
        client_secret=config.TWITCH_CLIENT_SECRET,
        auth_url=config.TWITCH_AUTH_URL,
        base_url=config.TWITCH_API_BASE_URL
    )
    # Trigger authentication early to check credentials
    if not api_client._authenticate():  # Use internal method carefully or add a public check method
        raise ConnectionError("Failed to authenticate with Twitch API.")
    print("API Client initialized and authenticated.")

except Exception as e:
    logging.critical(f"Initialization failed: {e}", exc_info=True)
    print(f"Initialization failed: {e}")
    if db_conn:
        db_conn.close()
    raise SystemExit("Stopping notebook due to initialization failure.")

print("-" * 30)
print(f"Setup Complete. Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("-" * 30)



In [ ]:
# Cell 2: Helper Functions

def parse_duration_for_collab(duration_str):
    """
    Parses a Twitch duration string (e.g., "1h30m20s") into total seconds.
    """
    if not duration_str or not isinstance(duration_str, str):
        return 0 # Return 0 for invalid input

    total_seconds = 0
    hours = re.search(r'(\d+)h', duration_str)
    minutes = re.search(r'(\d+)m', duration_str)
    seconds = re.search(r'(\d+)s', duration_str)

    if hours:
        total_seconds += int(hours.group(1)) * 3600
    if minutes:
        total_seconds += int(minutes.group(1)) * 60
    if seconds:
        total_seconds += int(seconds.group(1))

    return total_seconds

def format_seconds_to_hm(seconds):
    """
    Formats a duration in seconds into a human-readable "Xh Ym" string.
    """
    if pd.isna(seconds) or seconds < 0:
        return "N/A"

    seconds = int(seconds)
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60

    return f"{hours}h {minutes:02}m"

print("Helper functions defined.")

## Data Collection Seeding

This section is responsible for seeding the database with channels and their videos by fetching top streams and channels. It can be run to initialize an empty database, or to discover new channels that are not a part of the social network of any known channel.


In [ ]:
# Cell 2: Data Collection Cycle Function & Execution (Top Streams Focus)
def run_collection_cycle(current_api_client, current_db_conn):
    """
    Runs one cycle focused on fetching top streams, channels, and their videos.
    Mention processing is handled separately.
    """
    print(f"\n=== Starting Top Stream Data Collection Cycle at {datetime.now().strftime('%H:%M:%S')} ===")
    overall_start_time = time.time()

    # Phase 1: Fetch Top Categories
    phase_start_time = time.time()
    print("\n--- Phase 1: Fetching and Processing Top Categories ---")
    top_categories = current_api_client.get_top_games(config.NUM_TOP_CATEGORIES)
    if not top_categories: print("Could not fetch top categories. Cycle aborted."); return False
    database.save_categories(current_db_conn, top_categories)
    print(f"Phase 1: Processed {len(top_categories)} top categories in {time.time() - phase_start_time:.2f}s.")

    # Phase 2: Fetch Top Streams & Identify Channels
    phase_start_time = time.time()
    print("\n--- Phase 2: Fetching Top Streams & Identifying Channels from Categories ---")
    channels_to_process = set()
    categories_to_scan = database.get_categories_to_scan(current_db_conn, config.NUM_TOP_CATEGORIES)
    total_categories_to_scan = len(categories_to_scan)
    print(f"Found {total_categories_to_scan} categories prioritized for scanning.")
    category_processing_times = []

    for i, category_row in enumerate(categories_to_scan):
        cat_start_time = time.time()
        category_id = category_row['id']
        category_name = category_row['name']
        print(f" ({i + 1}/{total_categories_to_scan}) Processing category: {category_name}...")
        streams = current_api_client.get_streams_for_game(category_id, config.NUM_STREAMS_PER_CATEGORY)
        if streams:
            stream_channel_ids = set()
            for stream in streams:
                if 'user_id' in stream and 'user_login' in stream and 'user_name' in stream:
                    if database.save_channel_basic(current_db_conn, {
                        'id': stream['user_id'], 'login': stream['user_login'], 'display_name': stream['user_name']
                    }): stream_channel_ids.add(stream['user_id'])
            channels_to_process.update(stream_channel_ids)
        database.update_category_scan_time(current_db_conn, category_row['id'])

        cat_duration = time.time() - cat_start_time
        category_processing_times.append(cat_duration)
        if VERBOSE_MODE and category_processing_times and total_categories_to_scan > 0:
            avg_time_per_cat = sum(category_processing_times) / len(category_processing_times)
            est_remaining_cat_secs = (total_categories_to_scan - (i + 1)) * avg_time_per_cat
            if est_remaining_cat_secs > 0 and i < total_categories_to_scan - 1:
                est_cat_mins, est_cat_s = divmod(int(est_remaining_cat_secs), 60)
                print(f" -> Processed in {cat_duration:.2f}s. Est. remaining for categories: {est_cat_mins}m {est_cat_s}s")
            else:
                print(f" -> Processed in {cat_duration:.2f}s.")
        time.sleep(0.2)

    print(
        f"\nPhase 2: Identified {len(channels_to_process)} unique channels. Took {time.time() - phase_start_time:.2f}s.")

    # Phase 3: Fetch/Update Channel Details (MODIFIED for one-by-one fetching)
    phase_start_time = time.time()
    print("\n--- Phase 3: Fetching/Updating Channel Details (including Followers) ---")
    processed_channels_details = 0
    channels_needing_details_update = [
        chan_id for chan_id in list(channels_to_process)
        if database.check_channel_needs_update(current_db_conn, chan_id, config.REFETCH_CHANNEL_DETAILS_DAYS)
    ]
    total_to_update = len(channels_needing_details_update)
    print(f"{total_to_update} channels require detail fetching/updating.")
    detail_fetch_times = []

    if total_to_update > 0:
        for i, channel_id in enumerate(channels_needing_details_update):
            item_start_time = time.time()
            print(f" ({i + 1}/{total_to_update}) Fetching details for channel ID: {channel_id}...")

            # 1. Get user details (like login, description)
            user_details_list = current_api_client.get_user_details(user_ids=[channel_id])

            if user_details_list:
                user_data = user_details_list[0] # Get the first (and only) result

                # 2. Get follower count (separate API call)
                follower_count = current_api_client.get_channel_follower_count(broadcaster_id=channel_id)
                if follower_count is not None:
                    user_data['follower_count'] = follower_count

                # 3. Save combined data
                try:
                    database.save_channel_details(current_db_conn, user_data)
                    processed_channels_details += 1
                except Exception as e:
                    print(f" -> DB Error saving details for {user_data.get('login', channel_id)}: {e}")

            elif user_details_list is None:
                print(f" -> API call failed for details of channel {channel_id}. Skipping.")

            # Time estimation logic
            item_duration = time.time() - item_start_time
            detail_fetch_times.append(item_duration)
            if VERBOSE_MODE and detail_fetch_times:
                avg_time = sum(detail_fetch_times) / len(detail_fetch_times)
                est_rem_secs = (total_to_update - (i + 1)) * avg_time
                if est_rem_secs > 0:
                    est_mins, est_s = divmod(int(est_rem_secs), 60)
                    print(f" -> Processed in {item_duration:.2f}s. Est. remaining: {est_mins}m {est_s}s")

            time.sleep(0.3) # API courtesy between channels

    print(f"Phase 3: Finished. Attempted save for {processed_channels_details} channels. Took {time.time() - phase_start_time:.2f}s.")

    # Phase 4: Fetch/Update Channel Videos
    phase_start_time = time.time()
    print("\n--- Phase 4: Checking for and Fetching New Videos ---")
    channels_for_video_fetch = list(channels_to_process)
    total_channels_for_video = len(channels_for_video_fetch)
    print(f"Checking for new videos for {total_channels_for_video} channels from this cycle...")
    processed_channels_videos = 0; new_videos_found_total = 0
    video_fetch_times = []

    if total_channels_for_video > 0:
        for i, channel_id in enumerate(channels_for_video_fetch):
            ch_video_start_time = time.time()
            channel_info_for_log_cursor = current_db_conn.execute("SELECT login FROM Channels WHERE id = ?", (channel_id,))
            channel_info_for_log = channel_info_for_log_cursor.fetchone()
            channel_log_name = channel_info_for_log['login'] if channel_info_for_log else channel_id

            print(f" ({i + 1}/{total_channels_for_video}) Checking videos for channel: {channel_log_name}...")
            latest_stored_date = database.get_latest_video_date_for_channel(current_db_conn, channel_id)
            # This call fetches videos for channels just found in top streams
            new_videos = current_api_client.get_channel_videos(
                channel_id,
                video_type='archive',
                # Limit how many new videos to grab for a newly seen channel.
                # Helps prevent a single new channel from dominating the cycle time.
                limit=100,
                after_date=latest_stored_date
            )

            if new_videos:
                if VERBOSE_MODE: print(f" -> Found {len(new_videos)} new archive videos.")
                database.save_videos(current_db_conn, new_videos); new_videos_found_total += len(new_videos)
            elif new_videos is None:
                print(f" -> API call failed fetching videos for {channel_log_name}.")

            if new_videos is not None: database.update_channel_video_fetch_time(current_db_conn, channel_id)
            processed_channels_videos += 1

            ch_video_duration = time.time() - ch_video_start_time
            video_fetch_times.append(ch_video_duration)
            if VERBOSE_MODE and video_fetch_times:
                avg_time_per_ch_video = sum(video_fetch_times) / len(video_fetch_times)
                est_remaining_vid_secs = (total_channels_for_video - (i + 1)) * avg_time_per_ch_video
                if est_remaining_vid_secs > 0 and i < total_channels_for_video - 1:
                    est_vid_mins, est_vid_s = divmod(int(est_remaining_vid_secs), 60)
                    print(f" -> Processed in {ch_video_duration:.2f}s. Est. remaining for video checks: {est_vid_mins}m {est_vid_s}s")
                else:
                    print(f" -> Processed in {ch_video_duration:.2f}s.")
            time.sleep(0.5)

    print(f"\nPhase 4: Finished. Checked {processed_channels_videos} channels, found {new_videos_found_total} new videos. Took {time.time() - phase_start_time:.2f}s.")
    print(f"=== Top Stream Data Collection Cycle Finished in {time.time() - overall_start_time:.2f}s ===")
    return True

# Run the collection cycle (you can comment this out after initial runs or run it selectively)
print(f"\n--- Executing Data Collection Cycle (Top Streams) at {datetime.now().strftime('%H:%M:%S')} ---")
run_collection_cycle(api_client, db_conn)
print("-" * 30)
print("Data Collection Cycle (Top Streams) Done for this run.")
print("-" * 30)

## Mention Processing

This section processes videos to extract mentions (indicating collaborations) from their titles. New channels discovered through this process will be registered in the database, but we will not yet fetch these new channels' videos. Run this repeatedly until all videos in the database have been processed.


In [ ]:
# Cell 3: Mention Processing Function (Atomic Per-Video)

# Helper function to parse duration
def parse_duration_for_collab(duration_str):
    """
    Parses a Twitch duration string (e.g., "1h30m20s") into total seconds.
    """
    if not duration_str or not isinstance(duration_str, str):
        return 0 # Return 0 for invalid input

    total_seconds = 0
    hours = re.search(r'(\d+)h', duration_str)
    minutes = re.search(r'(\d+)m', duration_str)
    seconds = re.search(r'(\d+)s', duration_str)

    if hours:
        total_seconds += int(hours.group(1)) * 3600
    if minutes:
        total_seconds += int(minutes.group(1)) * 60
    if seconds:
        total_seconds += int(seconds.group(1))

    return total_seconds


def process_video_mentions_batch(video_batch, current_api_client, current_db_conn):
    """
    Processes a batch of videos to find mentions, update collaborations.
    Discovers new channels via API for unknown mentions.
    Attempts atomic processing per video using DB transactions.
    """
    func_start_time = time.time()
    processed_count_in_batch = 0;
    newly_found_channels_in_batch = 0;
    updated_edges_in_batch = 0;
    all_unknown_logins_in_batch = set();
    temp_video_data = {}

    if VERBOSE_MODE: print(f"  Batch Start: {len(video_batch)} videos to process.")

    # Pass 1: Extract mentions and identify all unique unknown logins for the batch
    for video_row in video_batch:
         video_id = video_row['id']; title = video_row['title'] or ''; desc = video_row['description'] or ''
         text_to_scan = f"{title} {desc}"
         mentioned_logins = network_utils.extract_mentions(text_to_scan)
         temp_video_data[video_id] = {
             'owner_id': video_row['channel_id'], 'mentions': mentioned_logins,
             'published_at': video_row['published_at'], 'duration': video_row['duration']
         }
         if mentioned_logins:
             try:
                 _, not_found_now = network_utils.find_mentioned_channel_ids(mentioned_logins, current_db_conn);
                 all_unknown_logins_in_batch.update(not_found_now)
             except Exception as e:
                 logging.error(f"Error checking mentions in DB during Pass 1 for video {video_id}: {e}")

    # Pass 2: Fetch details for unknown mentioned logins via batched API calls
    newly_discovered_ids_this_pass = {}
    if all_unknown_logins_in_batch:
        unknown_logins_list = list(all_unknown_logins_in_batch)
        if VERBOSE_MODE: print(f"  Pass 2: Checking {len(unknown_logins_list)} unique unknown logins via API...")
        num_api_batches = (len(unknown_logins_list) + 99) // 100
        for i in range(num_api_batches):
            batch_logins = unknown_logins_list[i * 100:(i + 1) * 100]
            if VERBOSE_MODE: print(f"   -> API Batch {i + 1}/{num_api_batches} for {len(batch_logins)} logins...")
            time.sleep(0.2)
            user_details_list = current_api_client.get_user_details(user_logins=batch_logins);
            api_call_succeeded = user_details_list is not None
            if api_call_succeeded and user_details_list:
                for user_data in user_details_list:
                     try:
                         database.save_channel_details(current_db_conn, user_data);
                         login_lower = user_data['login'].lower();
                         user_id = user_data['id']
                         newly_discovered_ids_this_pass[login_lower] = user_id;
                         newly_found_channels_in_batch += 1
                     except Exception as e:
                         print(f"Error saving newly discovered channel {user_data.get('login')}: {e}")
            time.sleep(0.1)
        if VERBOSE_MODE: print(f"  Pass 2 Complete. Discovered and saved {newly_found_channels_in_batch} new channels.")

    # Pass 3: Process each video within its own database transaction
    for idx, (video_id, video_data) in enumerate(temp_video_data.items()):
        channel_id_A = video_data['owner_id'];
        mentioned_logins = video_data['mentions']
        published_at = video_data['published_at'];
        duration_str = video_data['duration']
        is_processed_successfully_this_video = False

        try:
            published_at_dt = published_at
            if not isinstance(published_at_dt, datetime):
                published_at_dt = pd.to_datetime(published_at, errors='coerce', utc=True)

            if pd.isna(published_at_dt):
                logging.warning(f"Invalid timestamp for video {video_id}. Skipping edges. Marking processed.")
                current_db_conn.execute('BEGIN')
                database.mark_video_mentions_processed(current_db_conn, video_id)
                current_db_conn.commit()
                processed_count_in_batch += 1
                is_processed_successfully_this_video = True

            elif not mentioned_logins:
                current_db_conn.execute('BEGIN')
                database.mark_video_mentions_processed(current_db_conn, video_id)
                current_db_conn.commit()
                processed_count_in_batch += 1
                is_processed_successfully_this_video = True
            else:
                current_known_ids, _ = network_utils.find_mentioned_channel_ids(mentioned_logins, current_db_conn)
                for login, user_id_new in newly_discovered_ids_this_pass.items():
                     if login in mentioned_logins:
                         current_known_ids[login] = user_id_new

                duration_sec = parse_duration_for_collab(duration_str)
                edges_for_this_video = 0
                mentions_to_add_list = []

                current_db_conn.execute('BEGIN IMMEDIATE')
                for login, channel_id_B in current_known_ids.items():
                    if channel_id_A != channel_id_B:
                        database.upsert_collaboration_edge(current_db_conn, channel_id_A, channel_id_B, published_at_dt, duration_sec)
                        mentions_to_add_list.append((channel_id_A, channel_id_B, video_id, published_at_dt))
                        edges_for_this_video += 1

                if mentions_to_add_list:
                    database.add_mentions(current_db_conn, mentions_to_add_list)

                database.mark_video_mentions_processed(current_db_conn, video_id)
                current_db_conn.commit()

                processed_count_in_batch += 1
                updated_edges_in_batch += edges_for_this_video
                is_processed_successfully_this_video = True

        except Exception as e:
            if not is_processed_successfully_this_video:
                 try:
                     current_db_conn.rollback()
                 except sqlite3.Error as rb_err:
                     logging.error(f"Error during rollback for video {video_id}: {rb_err}")
                 logging.error(f"Error processing video {video_id} within transaction", exc_info=True)

    return processed_count_in_batch, newly_found_channels_in_batch, updated_edges_in_batch

In [ ]:
# Cell 4: Mention Processing Loop
print(f"\n--- Starting Mention Processing Phase at {datetime.now().strftime('%H:%M:%S')} ---")
BATCH_SIZE = config.MENTION_PROC_BATCH_SIZE
MAX_BATCHES_PER_RUN = config.MENTION_PROC_MAX_BATCHES
total_videos_processed_run = 0; total_new_channels_run = 0
total_edges_updated_run = 0; # Removed context counter
batches_processed_run = 0
mention_loop_start_time = time.time()
batch_processing_times_mention_loop = []

# Indication of total unprocessed videos at the START
try:
    unproc_cursor = db_conn.cursor()
    unproc_cursor.execute("SELECT COUNT(*) FROM Videos WHERE mentions_processed_at IS NULL")
    initial_total_unprocessed_videos = unproc_cursor.fetchone()[0]
    print(f"Estimated total unprocessed videos at start of run: {initial_total_unprocessed_videos}")
    if initial_total_unprocessed_videos > 0:
        initial_total_expected_batches = (initial_total_unprocessed_videos + BATCH_SIZE -1) // BATCH_SIZE
        print(f"Expecting around {initial_total_expected_batches} batches in total (this run will process up to {MAX_BATCHES_PER_RUN}).")
except sqlite3.Error as e_count:
    print(f"Could not get initial count of unprocessed videos: {e_count}")
    initial_total_expected_batches = MAX_BATCHES_PER_RUN

while batches_processed_run < MAX_BATCHES_PER_RUN:
    batch_loop_start_time = time.time()
    print(f"\nFetching mention processing batch {batches_processed_run + 1}/{MAX_BATCHES_PER_RUN} (Batch size: {BATCH_SIZE})...")
    videos_to_process = database.get_unprocessed_videos_batch(db_conn, BATCH_SIZE)

    if not videos_to_process: print("No more videos found needing mention processing."); break

    print(f"Processing mentions for {len(videos_to_process)} videos...")
    try:
        count, new_chans, edges = process_video_mentions_batch(videos_to_process, api_client, db_conn)

        total_videos_processed_run += count
        total_new_channels_run += new_chans
        total_edges_updated_run += edges
        batches_processed_run += 1
        batch_duration = time.time() - batch_loop_start_time
        batch_processing_times_mention_loop.append(batch_duration)

        print(f"Batch {batches_processed_run} complete in {batch_duration:.2f}s. Processed: {count} videos, Found: {new_chans} new channels, Upserted: {edges} edges.")

        if VERBOSE_MODE and batches_processed_run < MAX_BATCHES_PER_RUN and batch_processing_times_mention_loop and len(videos_to_process) == BATCH_SIZE:
            avg_time_per_batch = sum(batch_processing_times_mention_loop) / len(batch_processing_times_mention_loop)
            remaining_batches_in_run = MAX_BATCHES_PER_RUN - batches_processed_run
            batches_for_eta = remaining_batches_in_run
            if 'initial_total_expected_batches' in locals() and initial_total_expected_batches > batches_processed_run:
                 batches_for_eta = min(remaining_batches_in_run, initial_total_expected_batches - batches_processed_run)

            est_remaining_run_secs = batches_for_eta * avg_time_per_batch
            if est_remaining_run_secs > 0:
                est_run_mins, est_run_s = divmod(int(est_remaining_run_secs), 60)
                print(f"Est. time remaining for *this run* (up to {MAX_BATCHES_PER_RUN} batches, or fewer if DB empties): {est_run_mins}m {est_run_s}s")

        time.sleep(max(0.2, 1.0 - (0.05 * count / BATCH_SIZE if BATCH_SIZE > 0 else 1)))

    except Exception as e: print(f"Error during mention processing batch: {e}"); logging.error("Error in mention processing loop", exc_info=True); print("Stopping mention processing due to error."); break

# UPDATED: Removed context from final summary
print(f"\n--- Mention Processing Phase Finished (for this run) in {time.time() - mention_loop_start_time:.2f}s ---")
print(f"Total videos marked as processed in this run: {total_videos_processed_run}")
print(f"Total new channels discovered via mentions in this run: {total_new_channels_run}")
print(f"Total collaboration edge instances upserted in this run: {total_edges_updated_run}")

try:
    final_unproc_cursor = db_conn.cursor()
    final_unproc_cursor.execute("SELECT COUNT(*) FROM Videos WHERE mentions_processed_at IS NULL")
    final_unprocessed_videos = final_unproc_cursor.fetchone()[0]
    print(f"\nVideos remaining to be processed in database: {final_unprocessed_videos}")
except sqlite3.Error as e_count_end:
    print(f"Could not get final count of unprocessed videos: {e_count_end}")

print("-" * 30)

## Refresh Cycle

This section periodically updates details and recent videos for a prioritized subset of channels in the database. It focuses on channels that have not been updated recently ("stalest" channels), ensuring the dataset remains current without re-fetching all channels every cycle. The refresh cycle:

* Selects a configurable number of the stalest channels based on last update timestamps.
* Updates each channel's details (including follower count).
* Fetches and stores new videos for each refreshed channel.
* Helps maintain up-to-date collaboration and content data for ongoing analysis.

Run this cycle regularly to keep your database fresh and to discover new collaborations as channels continue to stream. Run it repeatedly until refreshes do not produce significant updates, at which point you can go back to the Mention Processing section.

In [ ]:
# Cell 5: Refresh Function

def run_refresh_cycle(current_api_client, current_db_conn, num_channels_to_refresh):
    """
    Refreshes details and fetches recent videos for a prioritized subset of
    the "stalest" channels (those not updated in the longest time). Includes follower count.
    """
    print(f"\n--- Starting Prioritized Channel Refresh Cycle at {datetime.now().strftime('%H:%M:%S')} ---")
    overall_refresh_start_time = time.time()
    processed_count = 0; new_videos_found_total = 0
    channel_refresh_times = []

    try:
        # Get prioritized list instead of random
        print(f"Fetching up to {num_channels_to_refresh} of the stalest channels from the database...")
        ids_to_refresh = database.get_stale_channels_for_refresh(current_db_conn, num_channels_to_refresh)
        actual_to_refresh_count = len(ids_to_refresh)

        if not ids_to_refresh:
            print("No channels found to refresh.")
            return

        print(f"Prioritized list created. Refreshing {actual_to_refresh_count} channels...")

        for i, channel_row in enumerate(ids_to_refresh):
            channel_refresh_start_time = time.time()
            time.sleep(0.4) # API courtesy
            channel_id = channel_row['id']
            channel_log_name = channel_row['login']

            print(f"\n ({i + 1}/{actual_to_refresh_count}) Refreshing channel: {channel_log_name}...")

            # 1. Refresh Channel Details (one-by-one)
            details_list = current_api_client.get_user_details(user_ids=[channel_id])

            if details_list:
                user_data = details_list[0]
                # Get follower count
                follower_count = current_api_client.get_channel_follower_count(broadcaster_id=channel_id)
                if follower_count is not None:
                    user_data['follower_count'] = follower_count

                try:
                    database.save_channel_details(current_db_conn, user_data)
                except Exception as e:
                    print(f"  -> DB Error saving details for channel {channel_id}: {e}")

            elif details_list is None:
                print(f"  -> API call failed fetching details for channel {channel_id}. Skipping.")
                continue # Skip this channel if we can't get its details

            # 2. Fetch New Videos
            if details_list is not None:
                latest_stored_date = database.get_latest_video_date_for_channel(current_db_conn, channel_id)

                new_videos = current_api_client.get_channel_videos(
                    channel_id,
                    video_type='archive',
                    # This limit prevents a single channel from dominating the refresh cycle.
                    # The database grows incrementally over multiple runs.
                    limit=100,
                    after_date=latest_stored_date
                )

                if new_videos:
                    if VERBOSE_MODE: print(f" -> Found {len(new_videos)} new archive videos.")
                    database.save_videos(current_db_conn, new_videos)
                    new_videos_found_total += len(new_videos)
                elif new_videos is None:
                    print(f" -> API call failed fetching videos for {channel_log_name}.")

                if new_videos is not None:
                    database.update_channel_video_fetch_time(current_db_conn, channel_id)

            processed_count += 1

            # Time estimation logic
            channel_refresh_duration = time.time() - channel_refresh_start_time
            channel_refresh_times.append(channel_refresh_duration)
            if VERBOSE_MODE and channel_refresh_times and actual_to_refresh_count > 0:
                avg_time_per_refresh = sum(channel_refresh_times) / len(channel_refresh_times)
                remaining_refreshes = actual_to_refresh_count - (i + 1)
                est_remaining_refresh_secs = remaining_refreshes * avg_time_per_refresh
                if est_remaining_refresh_secs > 0 and i < actual_to_refresh_count - 1:
                    est_ref_mins, est_ref_s = divmod(int(est_remaining_refresh_secs), 60)
                    print(f" -> Refreshed in {channel_refresh_duration:.2f}s. Est. time remaining for refresh cycle: {est_ref_mins}m {est_ref_s}s")
                else:
                    print(f" -> Refreshed in {channel_refresh_duration:.2f}s.")

        print(f"\n--- Channel Refresh Cycle Finished in {time.time() - overall_refresh_start_time:.2f}s ---")
        print(f"Attempted refresh for {processed_count} channels.")
        print(f"Found {new_videos_found_total} new videos in total during refresh.")

    except Exception as e:
        print(f"Error during refresh cycle: {e}")
        logging.error("Error in refresh cycle", exc_info=True)

In [ ]:
# Cell 6: Run Channel Refresh Cycle
print(f"\n--- Executing Channel Refresh Cycle at {datetime.now().strftime('%H:%M:%S')} ---")
run_refresh_cycle(api_client, db_conn, num_channels_to_refresh=config.REFRESH_CYCLE_CHANNELS)
print("-" * 30)
print("Channel Refresh Phase Done (for this run).")
print("-" * 30)


# Data Exploration

Load the collected data from the database into pandas DataFrames and perform basic analysis and visualization.


In [ ]:
# Cell 7: Load Data from Database
print(f"\n--- Loading Data for Exploration at {datetime.now().strftime('%H:%M:%S')} ---")
try:
    channel_date_cols = ['created_at', 'first_seen', 'last_fetched_details', 'last_fetched_videos']
    video_date_cols = ['published_at', 'created_at_api', 'fetched_at', 'mentions_processed_at']
    category_date_cols = ['last_scanned_top_streams']

    print("Loading Channels table...")
    channels_df = pd.read_sql_query("SELECT * FROM Channels", db_conn)
    print("Loading Videos table...")
    videos_df = pd.read_sql_query("SELECT * FROM Videos", db_conn)
    print("Loading Categories table...")
    categories_df = pd.read_sql_query("SELECT * FROM Categories", db_conn)

    # Convert timestamp columns
    print("Converting timestamp columns...")
    for col in channel_date_cols:
        if col in channels_df.columns: channels_df[col] = pd.to_datetime(channels_df[col], errors='coerce', utc=True)
    for col in video_date_cols:
        if col in videos_df.columns: videos_df[col] = pd.to_datetime(videos_df[col], errors='coerce', utc=True)
    for col in category_date_cols:
         if col in categories_df.columns: categories_df[col] = pd.to_datetime(categories_df[col], errors='coerce', utc=True)

    # Convert numeric columns, coercing errors
    print("Converting numeric columns and parsing durations...")
    channels_df['view_count'] = pd.to_numeric(channels_df['view_count'], errors='coerce')
    channels_df['follower_count'] = pd.to_numeric(channels_df['follower_count'], errors='coerce') # Add follower_count
    channels_df['display_name'] = channels_df['display_name'].fillna(channels_df['login'])
    videos_df['view_count'] = pd.to_numeric(videos_df['view_count'], errors='coerce')
    videos_df['duration_seconds'] = videos_df['duration'].apply(parse_duration_for_collab).fillna(0).astype(int)

    print(f"Loaded {len(channels_df)} channels, {len(videos_df)} videos, and {len(categories_df)} categories.")
    print("--- Data Loading Complete ---")


except Exception as e:
    print(f"Error loading data from database: {e}")
    raise SystemExit("Stopping notebook due to data loading failure.")

In [ ]:
# Cell 8: Display Sample Data
print("\nSample Channels Data:")
display(channels_df.head())
print("\nSample Videos Data:")
display(
    videos_df[['id', 'channel_id', 'title', 'published_at', 'view_count', 'duration_seconds', 'mentions_processed_at',
               'game_name']].head())
print("\nSample Categories Data:")
display(categories_df.head())


In [ ]:
# Cell 9: Basic Statistics
print("\n--- Basic Statistics ---")
print("\nChannel Statistics:")
print(f"Total Channels: {len(channels_df)}")
print("\nBroadcaster Types:")
print(channels_df['broadcaster_type'].value_counts(dropna=False))
print("\nChannel Follower Count Summary:")
print(channels_df['follower_count'].describe())
print(f"Channels missing details: {channels_df['last_fetched_details'].isnull().sum()}")

print("\nVideo Statistics:")
print(f"Total Videos: {len(videos_df)}")
print("\nVideo Types:")
print(videos_df['type'].value_counts(dropna=False))
print("\nVideo View Count Summary:")
print(videos_df['view_count'].describe())
print("\nVideo Duration (seconds) Summary:")
print(videos_df['duration_seconds'].describe())
print(f"Videos with mentions processed: {videos_df['mentions_processed_at'].notnull().sum()}")
print(f"Videos pending mention processing: {videos_df['mentions_processed_at'].isnull().sum()}")

## Data Visualization (Channels & Videos)


In [ ]:
# Cell 10: Visualizations - Channels
print("\n--- Channel Visualizations ---")
plt.style.use('seaborn-v0_8-whitegrid')

# --- UPDATED: Histogram of Channel Follower Counts ---
print("Generating Channel Follower Count Histogram...")
plt.figure(figsize=(10, 5));
# Use the 'follower_count' column now, and drop any rows where it might be NaN
followers_positive = channels_df.dropna(subset=['follower_count'])
followers_positive = followers_positive[followers_positive['follower_count'] > 0]['follower_count']

if not followers_positive.empty:
    # Determine if log scale is needed by checking the data range
    should_use_log = (followers_positive.max() / followers_positive.min() > 100) if followers_positive.min() > 0 else False

    # Conditionally set the plotting arguments
    plot_kwargs = {'bins': 40, 'kde': False}
    if should_use_log:
        plot_kwargs['log_scale'] = True # Set to True, not a variable that could be False
        title = 'Distribution of Channel Follower Counts (Log Scale)'
        xlabel = 'Total Follower Count (Log Scale)'
    else:
        # Do not pass log_scale when it's not needed
        title = 'Distribution of Channel Follower Counts'
        xlabel = 'Total Follower Count'

    sns.histplot(followers_positive, **plot_kwargs)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('Number of Channels')
    plt.tight_layout()
    plt.show()
else:
    print("No positive channel follower data to plot histogram yet.")

print("Generating Broadcaster Types Bar Chart...")
plt.figure(figsize=(8, 5));
channel_types = channels_df['broadcaster_type'].fillna('N/A').replace('', 'N/A')
sns.countplot(y=channel_types, order=channel_types.value_counts().index, palette='viridis');
plt.title('Channel Broadcaster Types')
plt.xlabel('Number of Channels')
plt.ylabel('Broadcaster Type')
plt.tight_layout()
plt.show()

In [ ]:
# Cell 11: Visualizations - Videos
print("\n--- Video Visualizations ---")

print("Generating Video View Count Histogram...")
plt.figure(figsize=(10, 5));
views_positive_vid = videos_df.dropna(subset=['view_count'])
views_positive_vid = views_positive_vid[views_positive_vid['view_count'] > 0]['view_count']

if not views_positive_vid.empty:
    # Determine if log scale is needed
    should_use_log_vid = (views_positive_vid.max() / views_positive_vid.min() > 100) if views_positive_vid.min() > 0 else False

    # Conditionally set the plotting arguments
    plot_kwargs_vid = {'bins': 40, 'kde': False}
    if should_use_log_vid:
        plot_kwargs_vid['log_scale'] = True
        title_vid = 'Distribution of Video View Counts (Log Scale)'
        xlabel_vid = 'Video View Count (Log Scale)'
    else:
        title_vid = 'Distribution of Video View Counts'
        xlabel_vid = 'Video View Count'

    sns.histplot(views_positive_vid, **plot_kwargs_vid)
    plt.title(title_vid)
    plt.xlabel(xlabel_vid)
    plt.ylabel('Number of Videos')
    plt.tight_layout()
    plt.show()
else:
    print("No positive video view data to plot histogram.")

print("Generating Video Types Bar Chart...")
plt.figure(figsize=(8, 5));
video_types = videos_df['type'].fillna('N/A');
sns.countplot(y=video_types, order=video_types.value_counts().index, palette='magma');
plt.title('Video Types');
plt.xlabel('Number of Videos');
plt.ylabel('Type');
plt.tight_layout();
plt.show()

# --- Video Publication Time Series ---
print("Generating Video Publication Time Series...")
plt.figure(figsize=(12, 6));
video_pub_dates = videos_df.dropna(subset=['published_at'])

if not video_pub_dates.empty and len(video_pub_dates) > 1:
    # --- Calculate the 5th percentile date to set the start range ---
    start_date = video_pub_dates['published_at'].quantile(0.05)
    print(f"Displaying time series from {start_date.strftime('%Y-%m-%d')} onwards (showing 95% of the data).")

    # Filter the DataFrame to this new date range
    plotting_df = video_pub_dates[video_pub_dates['published_at'] >= start_date]

    # --- Use the filtered plotting_df for the rest of the logic ---
    if not plotting_df.empty:
        time_range_days = (plotting_df['published_at'].max() - plotting_df['published_at'].min()).days if len(plotting_df) > 1 else 0
        resample_freq = 'ME' if time_range_days > 90 else 'D' # Resample by Month or Day
        plot_title = 'Number of Videos Published (' + ('Monthly' if resample_freq == 'ME' else 'Daily') + ')'

        # Plot the resampled data
        plotting_df.set_index('published_at')['id'].resample(resample_freq).count().plot(marker='.' if resample_freq == 'D' else 'o', linestyle='-');

        plt.title(plot_title);
        plt.ylabel('Number of Videos');
        plt.xlabel('Publication Date');
        plt.tight_layout();
        plt.show()
    else:
        print("No video data remains after filtering by start date.")
else:
    print("Not enough video publication date data to plot a meaningful time series.")

### Note on "Streaming Together" Feature

While Twitch has features like "Squad Streams" or "Guest Star" allowing multiple creators to stream simultaneously on one channel, the participant data for these features **does not appear to be reliably available** via the standard Twitch API for *past* streams or VODs (as of June 2025).

Therefore, the collaboration detection in this notebook relies primarily on identifying `@mentions` within video titles and descriptions.


# Collaboration Network Exploration (Filtered)

Exploring the collaboration network. Data is filtered IN MEMORY based on
thresholds in `config.py` BEFORE analysis to improve performance.
The underlying database remains complete.


In [ ]:
# Cell 12: Load and Filter Data for Network Analysis
print(f"\n--- Loading and Filtering Data for Network Analysis at {datetime.now().strftime('%H:%M:%S')} ---")
channels_df_net = pd.DataFrame()  # For channels passing all filters
collab_df_net = pd.DataFrame()   # For edges passing all filters
G_filtered = nx.Graph()          # The filtered graph for analysis
degree_df_filtered = pd.DataFrame() # For degree stats

try:
    # --- 1. Load base data ---
    collab_df_full = pd.read_sql_query("SELECT * FROM Collaborations", db_conn)
    collab_df_full['collaboration_count'] = pd.to_numeric(collab_df_full['collaboration_count'], errors='coerce').fillna(0).astype(int)
    collab_df_full['total_collaboration_duration_seconds'] = pd.to_numeric(collab_df_full['total_collaboration_duration_seconds'], errors='coerce').fillna(0).astype(int)


    # --- 2. Filter Channels ---
    print(f"Filtering channels by follower count (>= {config.NETWORK_MIN_FOLLOWER_COUNT})...")
    channels_df_filtered_fc = channels_df[channels_df['follower_count'] >= config.NETWORK_MIN_FOLLOWER_COUNT]
    print(f" -> Channels after follower count filter: {len(channels_df_filtered_fc)}")

    if not videos_df.empty:
        print(f"Filtering channels by video count (>= {config.NETWORK_MIN_CHANNEL_VIDEO_COUNT})...")
        video_counts_per_channel = videos_df['channel_id'].value_counts()
        channels_with_enough_videos = video_counts_per_channel[video_counts_per_channel >= config.NETWORK_MIN_CHANNEL_VIDEO_COUNT].index.tolist()
        channels_df_net = channels_df_filtered_fc[channels_df_filtered_fc['id'].isin(channels_with_enough_videos)]
        print(f" -> Channels after video count filter: {len(channels_df_net)}")
    else:
        print("Warning: videos_df not available for filtering by video count. Using only follower count filter for channels.")
        channels_df_net = channels_df_filtered_fc

    valid_channel_ids_for_network = set(channels_df_net['id'])
    print(f"Total channels passing node filters: {len(valid_channel_ids_for_network)}")

    # --- 3. Filter Collaborations (Edges) ---
    if not collab_df_full.empty:
        print(f"Filtering collaboration edges by count (>= {config.NETWORK_MIN_COLLABORATION_COUNT})...")
        collab_df_filtered_count = collab_df_full[collab_df_full['collaboration_count'] >= config.NETWORK_MIN_COLLABORATION_COUNT]
        print(f" -> Edges after count filter: {len(collab_df_filtered_count)}")

        print("Filtering edges to ensure both connected channels passed node filters...")
        collab_df_net = collab_df_filtered_count[
            collab_df_filtered_count['channel_id_1'].isin(valid_channel_ids_for_network) &
            collab_df_filtered_count['channel_id_2'].isin(valid_channel_ids_for_network)
        ].copy() # Use .copy() to avoid SettingWithCopyWarning
        print(f" -> Edges after node validity filter: {len(collab_df_net)}")

        # --- Cap outlier durations before creating graph ---
        duration_threshold_weeks = config.NETWORK_DURATION_OUTLIER_WEEKS
        duration_threshold_seconds = duration_threshold_weeks * 7 * 24 * 3600

        outlier_edges = collab_df_net['total_collaboration_duration_seconds'] > duration_threshold_seconds
        outlier_count = outlier_edges.sum()

        if outlier_count > 0:
            # The f-string now uses the config variable directly for accurate reporting
            print(f"Capping duration for {outlier_count} edge(s) with duration > {duration_threshold_weeks} week(s) for network analysis.")

            # Use .loc to safely modify the DataFrame
            collab_df_net.loc[outlier_edges, 'total_collaboration_duration_seconds'] = duration_threshold_seconds

    else:
        print("Full collaboration data (collab_df_full) is empty. Filtered collaboration data will be empty.")
        collab_df_net = pd.DataFrame()

    # --- 4. Create Filtered NetworkX Graph ---
    if not collab_df_net.empty:
        print("Creating NetworkX graph G_filtered from filtered data...")
        G_filtered = nx.from_pandas_edgelist(
            collab_df_net,
            'channel_id_1',
            'channel_id_2',
            edge_attr=['collaboration_count', 'total_collaboration_duration_seconds', 'latest_collaboration_timestamp']
        )
        G_filtered.add_nodes_from(valid_channel_ids_for_network)
        print(f"Filtered graph G_filtered created with {G_filtered.number_of_nodes()} nodes and {G_filtered.number_of_edges()} edges.")
    else:
        print("No edges passed all filters. Filtered graph G_filtered will be empty or contain only isolated nodes.")
        if valid_channel_ids_for_network: G_filtered.add_nodes_from(valid_channel_ids_for_network)
        print(f"Filtered graph G_filtered has {G_filtered.number_of_nodes()} nodes and {G_filtered.number_of_edges()} edges.")

except Exception as e:
    print(f"Error loading or filtering data for network analysis: {e}")
    logging.error("Error in network data prep:", exc_info=True)
    if 'G_filtered' not in locals(): G_filtered = nx.Graph()

print("--- Network Data Preparation Complete ---")

In [ ]:
# Cell 13: Display Sample Filtered Collaboration Data
print("\nSample Filtered Collaboration Edges (collab_df_net):")
if not collab_df_net.empty:
    display(collab_df_net.head())
else:
    print("No collaboration data in collab_df_net (all edges filtered out or none exist).")


In [ ]:
# Cell 14: Filtered Collaboration Network Statistics
if 'G_filtered' in locals() and G_filtered.number_of_nodes() > 0:
    print("\n--- Filtered Collaboration Network Statistics ---")
    print(f"Total Edges in Filtered Network: {G_filtered.number_of_edges()}")
    print(f"Total Nodes in Filtered Network: {G_filtered.number_of_nodes()}")

    if G_filtered.number_of_nodes() > 0:
        degree_sequence = [d for n, d in G_filtered.degree()]
        degree_df_filtered = pd.DataFrame({'channel_id': list(G_filtered.nodes()), 'degree': degree_sequence})

        print("\nDegree Distribution Summary (Filtered Network):")
        print(degree_df_filtered['degree'].describe())

        # Merge with channel names for context
        channels_for_labels_df = channels_df[['id', 'login', 'display_name']].rename(columns={'id': 'channel_id'})
        degree_df_filtered = pd.merge(degree_df_filtered, channels_for_labels_df, on='channel_id', how='left')

        print("\nTop 10 Channels by Degree (Filtered Network):")
        print(degree_df_filtered.nlargest(10, 'degree'))
    else:
        print("Filtered graph has no nodes to calculate degree.")

    # Summary for edges in collab_df_net
    if not collab_df_net.empty:
        print("\nCollaboration Count per Edge Summary (Filtered Network):")
        print(collab_df_net['collaboration_count'].describe())

        # --- Human-Readable Duration Summary ---
        print("\nTotal Collaboration Duration per Edge Summary (Filtered Network):")
        duration_stats = collab_df_net['total_collaboration_duration_seconds'].describe()

        # Create and print a formatted summary
        formatted_stats = {
            'count': f"{duration_stats['count']:.0f}",
            'mean': format_seconds_to_hm(duration_stats['mean']),
            'std dev': f"~{format_seconds_to_hm(duration_stats['std'])}",
            'min': format_seconds_to_hm(duration_stats['min']),
            '25%': format_seconds_to_hm(duration_stats['25%']),
            '50% (median)': format_seconds_to_hm(duration_stats['50%']),
            '75%': format_seconds_to_hm(duration_stats['75%']),
            'max': format_seconds_to_hm(duration_stats['max'])
        }

        for key, value in formatted_stats.items():
            # Left-align the key, right-align the value for clean output
            print(f"{key:<15} {value:>15}")

    else:
        print("No edges in collab_df_net to summarize.")
else:
    print("\nFiltered graph G_filtered is empty. No statistics to display.")

In [ ]:
# Cell 15: Filtered Collaboration Network Visualizations
if 'G_filtered' in locals() and G_filtered.number_of_nodes() > 0 and 'degree_df_filtered' in locals() and not degree_df_filtered.empty:
    print("\n--- Filtered Collaboration Network Visualizations ---")

    # --- 1. Degree Distribution Histogram ---
    print("Generating Degree Distribution Histogram for the filtered network...")
    plt.figure(figsize=(10, 6))

    degrees_filtered = degree_df_filtered[degree_df_filtered['degree'] > 0]['degree']
    if not degrees_filtered.empty:
        # Log-log scale is common for degree distributions to check for power-law behavior
        sns.histplot(degrees_filtered, log_scale=True, bins=30)
        plt.title('Degree Distribution of Filtered Collaboration Network (Log-Log Scale)')
        plt.xlabel('Degree (Number of Unique Collaborators) - Log Scale')
        plt.ylabel('Number of Channels - Log Scale')
        plt.tight_layout()
        plt.show()
    else:
        print("No nodes with degree > 0 found to plot.")

    # --- 2. Edge Weight (Collaboration Count) Plot ---
    if not collab_df_net.empty:
        print("\nGenerating Edge Weight (Collaboration Count) Plot...")
        plt.figure(figsize=(10, 6))

        collab_counts_filtered = collab_df_net[collab_df_net['collaboration_count'] > 0]['collaboration_count']

        if not collab_counts_filtered.empty:
            # A countplot is more direct and robust for this type of discrete integer data
            ax = sns.countplot(x=collab_counts_filtered, palette='viridis', order = sorted(collab_counts_filtered.unique()))

            # Add text labels on top of each bar
            for p in ax.patches:
                ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha='center', va='center', xytext=(0, 5), textcoords='offset points')

            plt.title('Distribution of Collaboration Counts per Edge (Filtered Network)')
            plt.xlabel('Number of Collaborations on a Single Edge')

            # Conditionally apply log scale to y-axis only if counts are high
            max_count = collab_counts_filtered.value_counts().max()
            if max_count > 10:
                plt.yscale('log')
                plt.ylabel('Number of Edges (Log Scale)')
            else:
                plt.ylabel('Number of Edges (Linear Scale)')

            plt.tight_layout()
            plt.show()
        else:
            print("No collaborations with count > 0 found to plot.")

else:
    print("\nFiltered graph or degree data insufficient for visualization. Skipping.")

## Community Detection in Collaboration Network

In [ ]:
# Cell 16: Community Detection Setup and Execution
import community as community_louvain  # python-louvain library

communities_detected = False
partition = {}
communities = {}

if 'G_filtered' in locals() and G_filtered.number_of_nodes() > 0:
    print(
        f"\n--- Performing Louvain Community Detection on FILTERED graph at {datetime.now().strftime('%H:%M:%S')} ---")
    print(f"Using 'collaboration_count' as edge weight for community detection.")

    for u, v, data in G_filtered.edges(data=True):
        if 'weight' not in data:
            G_filtered.edges[u, v]['weight'] = data.get('collaboration_count', 1)

    start_time_community = time.time()
    try:
        partition = community_louvain.best_partition(G_filtered, weight='weight', random_state=42)
        modularity = community_louvain.modularity(partition, G_filtered, weight='weight')

        for node, community_id in partition.items():
            if community_id not in communities: communities[community_id] = []
            communities[community_id].append(node)

        num_communities = len(communities)
        print(f"Louvain Community Detection complete in {time.time() - start_time_community:.2f}s.")
        print(f"Found {num_communities} communities.")
        print(f"Modularity of the partition: {modularity:.4f}")

        sorted_communities_by_size = sorted(communities.items(), key=lambda item: len(item[1]), reverse=True)
        print("\nTop 5 Largest Communities:")
        for i in range(min(5, len(sorted_communities_by_size))):
            cid, nodes = sorted_communities_by_size[i]
            member_names = [
                channels_for_labels_df.loc[channels_for_labels_df['channel_id'] == node_id, 'login'].iloc[0]
                for node_id in nodes[:3]
                if not channels_for_labels_df[channels_for_labels_df['channel_id'] == node_id].empty
            ]
            print(
                f"  Community {cid} (Size rank {i + 1}): {len(nodes)} members. Examples: {', '.join(member_names[:3])}...")
        communities_detected = True
    except Exception as e:
        print(f"Error during community detection: {e}")
else:
    print("\nFiltered graph G_filtered not found or is empty. Skipping community detection.")


### Visualizing Communities

In [ ]:
# %%
# Cell 17: Visualize Subgraph with Community Colors
if communities_detected and 'G_filtered' in locals() and G_filtered.number_of_nodes() > 0:
    print("\n--- NetworkX Subgraph Visualization with Community Colors ---")
    try:
        # Use the same subgraph logic as before to select nodes for visualization
        top_nodes_filtered = degree_df_filtered.nlargest(config.NETWORK_VIZ_TOP_N_CHANNELS_BY_DEGREE, 'degree')['channel_id'].tolist()
        subgraph_nodes_set = set(top_nodes_filtered)
        max_subgraph_nodes = config.NETWORK_VIZ_MAX_SUBGRAPH_NODES

        for node in list(subgraph_nodes_set):
            if G_filtered.has_node(node) and len(subgraph_nodes_set) < max_subgraph_nodes:
                neighbors = list(G_filtered.neighbors(node))
                needed = max_subgraph_nodes - len(subgraph_nodes_set)
                subgraph_nodes_set.update(neighbors[:needed])

        subgraph = G_filtered.subgraph(list(subgraph_nodes_set))
        print(f"Creating subgraph visualization with {subgraph.number_of_nodes()} nodes and {subgraph.number_of_edges()} edges, clustered by community.")

        if subgraph.number_of_nodes() > 0:
            # --- NEW: Create a temporary graph for community-aware layout ---
            # We will modify edge weights in this copy to influence the layout
            layout_graph = subgraph.copy()

            # Increase weight for intra-community edges, decrease for inter-community
            # This makes the "springs" inside a community much stronger.
            for u, v in layout_graph.edges():
                if partition.get(u) == partition.get(v):
                    # Strengthen connection if nodes are in the same community
                    layout_graph.edges[u,v]['weight'] = 5  # High weight for strong attraction
                else:
                    # Weaken connection if nodes are in different communities
                    layout_graph.edges[u,v]['weight'] = 0.1 # Low weight for weak attraction

            # --- Calculate positions using the modified layout_graph ---
            pos_subgraph = nx.spring_layout(layout_graph, weight='weight', k=0.4, iterations=50, seed=42)

            # --- Visualization logic remains mostly the same, but uses the new positions ---

            # Get community colors for nodes in the subgraph
            unique_community_ids_in_subgraph = sorted(list(set(partition[node] for node in subgraph.nodes() if node in partition)))
            community_to_color_idx = {comm_id: i for i, comm_id in enumerate(unique_community_ids_in_subgraph)}
            num_distinct_colors_needed = len(unique_community_ids_in_subgraph)

            node_colors_subgraph = 'grey'
            if num_distinct_colors_needed > 0:
                cmap = plt.cm.get_cmap('tab20', num_distinct_colors_needed) if num_distinct_colors_needed <= 20 else plt.cm.get_cmap('viridis', num_distinct_colors_needed)
                node_colors_subgraph = [cmap(community_to_color_idx.get(partition.get(node), -1)) for node in subgraph.nodes()]

            # Get node sizes and labels based on the original data
            node_data_subgraph = channels_df[channels_df['id'].isin(subgraph.nodes())].set_index('id')
            subgraph_node_sizes = [math.log10(max(1, node_data_subgraph.loc[node, 'follower_count'] if node in node_data_subgraph.index else 1)+1) * 300 + 100 for node in subgraph.nodes()]
            node_labels_subgraph = pd.merge(pd.DataFrame({'channel_id': list(subgraph.nodes())}), channels_for_labels_df, on='channel_id', how='left').set_index('channel_id')['login'].to_dict()

            # Use original collaboration count for visual edge thickness
            edge_widths_subgraph = [math.log10(max(0, d.get('collaboration_count',0)) + 1) * 1.5 + 0.5 for u, v, d in subgraph.edges(data=True)]

            # --- Drawing ---
            plt.figure(figsize=(18, 15))

            # Draw using the original subgraph data but the new positions (pos_subgraph)
            nx.draw_networkx_nodes(subgraph, pos_subgraph, node_size=subgraph_node_sizes, node_color=node_colors_subgraph, alpha=0.9)
            nx.draw_networkx_edges(subgraph, pos_subgraph, width=edge_widths_subgraph, alpha=0.2, edge_color='gray')
            nx.draw_networkx_labels(subgraph, pos_subgraph, labels=node_labels_subgraph, font_size=8)

            plt.title(f'Collaboration Subgraph with Community Colors (Clustered Layout)', fontsize=16)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
        else:
            print("Subgraph is empty, cannot visualize communities.")

    except ImportError:
        print("\n`python-louvain` (community) library not found. Skipping community detection visualization.")
    except Exception as e:
        print(f"Error during community visualization: {e}")
        logging.error("Community visualization error", exc_info=True)

else:
    print("\nNo communities detected or filtered graph is empty. Skipping community visualization.")

# Collaboration Topic Modeling with BERTopic

Here, we derive context by analyzing the content of video titles. We use BERTopic to automatically discover topics from the titles of videos belonging to channels in our filtered collaboration network. This helps us understand what collaborating streamers talk about or play, without relying on pre-defined categories.


In [ ]:
# %%
# Cell: BERTopic Pre-processing: Assign Topic ID to All Relevant Videos

# This cell trains the BERTopic model on a large sample of video titles
# and then assigns a topic ID to each video in a new DataFrame.
# This is a time-consuming step that enables fast context lookups later.

# Initialize global-like variables to hold the results
topic_model = None
videos_with_topics_df = None

print("\n--- Starting Collaboration Topic Modeling with BERTopic ---")

# --- 1. Main Gate: Check if we have a filtered graph to work with ---
if 'G_filtered' not in locals() or G_filtered.number_of_nodes() == 0:
    print("\nFiltered graph G_filtered not found or is empty. Skipping topic modeling.")

else:
    # --- 2. If graph exists, proceed with data preparation and modeling ---
    try:
        from bertopic import BERTopic
        import nltk
        from nltk.corpus import stopwords
        import re

        # Setup NLTK stopwords
        try:
            stopwords.words('english')
        except LookupError:
            print("NLTK stopwords not found. Downloading...")
            nltk.download('stopwords')

        stop_words = set(stopwords.words('english'))
        mention_regex = re.compile(r'@([a-zA-Z0-9_]{4,25})')

        def clean_title(title):
            if not isinstance(title, str): return ""
            text = mention_regex.sub('', title)
            text = re.sub(r'[^A-Za-z0-9\s]+', '', text)
            text = text.lower()
            text = ' '.join([word for word in text.split() if word not in stop_words])
            return text

        print("Preparing data for topic modeling...")
        nodes_in_network = list(G_filtered.nodes())
        videos_for_modeling_df = videos_df[videos_df['channel_id'].isin(nodes_in_network)].copy()
        print(f"Found {len(videos_for_modeling_df)} videos from {len(nodes_in_network)} channels in the filtered network.")

        print(f"Filtering for English language videos...")
        english_videos_df = videos_for_modeling_df[videos_for_modeling_df['language'] == 'en'].copy()
        print(f" -> Found {len(english_videos_df)} English videos to model.")

        print("Cleaning video titles (removing stopwords and mentions)...")
        english_videos_df['cleaned_title'] = english_videos_df['title'].dropna().apply(clean_title)

        docs_df = english_videos_df[english_videos_df['cleaned_title'].str.len() > 0].copy()
        docs = docs_df['cleaned_title'].tolist()

        # --- 3. Check if we have enough documents to proceed ---
        if len(docs) < 50:
            print(f"Not enough video titles found ({len(docs)}) after cleaning to perform topic modeling.")

        else:
            # --- 4. This is where the actual modeling happens ---
            MAX_DOCS_FOR_MODELING = 50000
            if len(docs) > MAX_DOCS_FOR_MODELING:
                print(f"Sampling {MAX_DOCS_FOR_MODELING} titles for topic modeling to ensure performance...")
                docs_df = docs_df.sample(n=MAX_DOCS_FOR_MODELING, random_state=42)
                docs = docs_df['cleaned_title'].tolist()

            print(f"Training BERTopic model on {len(docs)} cleaned video titles. This may take several minutes...")
            print("(The first run will download pre-trained language models).")

            topic_model = BERTopic(
                min_topic_size=20,
                verbose=True
            )

            topics, _ = topic_model.fit_transform(docs)

            print("Mapping topic results back to videos...")
            docs_df['topic_id'] = topics
            videos_with_topics_df = docs_df # Assign to the main variable for other cells to use

            print("\n--- BERTopic Analysis Complete ---")

            print("\nTop discovered topics from cleaned titles:")
            display(topic_model.get_topic_info())

            print("\nVisualizing top topics (barchart):")
            display(topic_model.visualize_barchart(top_n_topics=10))

            print("\nVisualizing inter-topic distance map:")
            display(topic_model.visualize_topics())

    # --- 5. Handle potential errors for the entire block ---
    except ImportError:
        print("\nCould not perform topic modeling. Please install required libraries:")
        print("pip install bertopic sentence-transformers scikit-learn nltk")
    except Exception as e:
        print(f"An error occurred during topic modeling: {e}")
        logging.error("BERTopic analysis failed", exc_info=True)

In [ ]:
# Cell: Diagnostic for BERTopic Results

if 'topic_model' in locals() and topic_model is not None:
    print("--- BERTopic Model Diagnostics ---")

    # Get the overview of topics
    topic_info = topic_model.get_topic_info()

    # Check the size of the outlier topic (-1)
    outlier_info = topic_info[topic_info['Topic'] == -1]
    if not outlier_info.empty:
        total_docs = topic_info['Count'].sum()
        outlier_count = outlier_info['Count'].iloc[0]
        outlier_percentage = (outlier_count / total_docs) * 100
        print(f"\nOutlier Topic (-1) contains {outlier_count} videos ({outlier_percentage:.1f}% of the total).")

    print("\nOverview of the top 10 most frequent topics (excluding outliers):")
    display(topic_info[topic_info['Topic'] != -1].head(10))

else:
    print("BERTopic model has not been trained yet.")

# Interactive Collaboration Network Snippet (Searchable)

Type part of a channel name (login or display name) and click "Search & Visualize" to see its immediate collaboration network.

- If multiple channels match your search, select the specific one from the second dropdown.
- Node size is proportional to the channel's total follower count (log scale).
- Edge thickness is proportional to the number of collaboration instances found (log scale).
- Edge labels show the most frequent game category for the collaboration and its percentage.

*(Note: Embedding channel profile pictures directly within nodes is complex with this plotting method and is omitted.)*

In [ ]:
# Cell: Setup for Searchable Interactive Visualization
from ipywidgets import Text, Button, Dropdown, Output, VBox, HBox, Layout
from IPython.display import display
import networkx as nx
import matplotlib.pyplot as plt
import math
import pandas as pd
import sqlite3

# This variable will hold the ID of the last channel successfully visualized
# so we can re-plot when the degree of separation is changed.
currently_visualized_channel_id = None

# --- Ensure data is loaded from previous cells ---
# This check ensures that if you only run the bottom part of the notebook, data is available.
if 'channels_df' not in locals() or channels_df.empty:
    print("Warning: Full 'channels_df' not loaded. Reloading for search widget...")
    try:
        channels_df = pd.read_sql_query("SELECT id, login, display_name, follower_count FROM Channels", db_conn)
        channels_df['follower_count'] = pd.to_numeric(channels_df['follower_count'], errors='coerce').fillna(1)
        channels_df['display_name'] = channels_df['display_name'].fillna(channels_df['login'])
    except Exception as e:
        print(f"Failed to reload channel data: {e}")
        channels_df = pd.DataFrame(columns=['id', 'login', 'display_name', 'follower_count'])

if 'collab_df_full' not in locals() or collab_df_full.empty:
    print("Warning: Full collaboration data not loaded. Reloading...")
    try:
        collab_df_full = pd.read_sql_query("SELECT * FROM Collaborations", db_conn)
        collab_df_full['collaboration_count'] = pd.to_numeric(collab_df_full['collaboration_count'], errors='coerce').fillna(0).astype(int)
    except Exception as e:
        print(f"Failed to reload collaboration data: {e}")
        collab_df_full = pd.DataFrame(columns=['channel_id_1', 'channel_id_2', 'collaboration_count'])

if 'videos_with_topics_df' not in locals():
     print("Warning: BERTopic data ('videos_with_topics_df') not found. Edge labels will be empty.")
     videos_with_topics_df = None # Ensure variable exists to prevent errors

# --- Create Widgets ---
search_input = Text(
    description="Channel Name:",
    placeholder="Enter login or display name (min 3 chars)",
    style={'description_width': 'initial'}
)

search_button = Button(
    description="Search",
    button_style='info',
    tooltip='Search for the channel and display its network',
)

# --- NEW: Dropdown for degrees of separation ---
degree_selector = Dropdown(
    options=[
        ('1 (Direct Neighbors)', 1),
        ('2 (Neighbors of Neighbors)', 2),
        ('3', 3),
        ('4', 4)
    ],
    value=1, # Default to 1 degree
    description='Degrees:',
    style={'description_width': 'initial'}
)

results_dropdown = Dropdown(
    description="Select Match:",
    options=[("---", None)],
    disabled=True,
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)

message_output = Output(layout={'margin': '10px 0 0 0'})
plot_output = Output()

In [ ]:
# Cell: Helper function for BERTopic Context

def get_shared_topic_context(channel_id_A, channel_id_B, videos_with_topics, topic_model, top_n=1):
    """
    Infers a shared context between two channels based on the dominant topics
    of their video titles, using the pre-generated CustomName or keywords.
    """
    # Check if the BERTopic model and results are available
    if videos_with_topics is None or topic_model is None:
        return "" # Not ready yet

    try:
        # Get topics for each channel's videos from the pre-processed DataFrame
        videos_A = videos_with_topics[videos_with_topics['channel_id'] == channel_id_A]
        videos_B = videos_with_topics[videos_with_topics['channel_id'] == channel_id_B]

        if videos_A.empty or videos_B.empty:
            return "" # Not enough video data for one of the channels

        # Get the frequency of each topic for each channel (ignoring outlier topic -1)
        freq_A = videos_A[videos_A['topic_id'] != -1]['topic_id'].value_counts(normalize=True)
        freq_B = videos_B[videos_B['topic_id'] != -1]['topic_id'].value_counts(normalize=True)

        if freq_A.empty or freq_B.empty:
            return "" # No non-outlier topics found

        # Find common topics and calculate a simple "shared interest" score by multiplying their frequencies
        common_topics = freq_A.index.intersection(freq_B.index)

        if len(common_topics) == 0:
            return "" # No topics in common

        shared_scores = freq_A[common_topics] * freq_B[common_topics]

        # Get the top N shared topics based on this score
        top_shared_topic_ids = shared_scores.nlargest(top_n).index

        labels = []
        for topic_id in top_shared_topic_ids:
            # Check if a CustomName was generated by an LLM
            if 'CustomName' in videos_with_topics.columns:
                topic_name_row = videos_with_topics[videos_with_topics['topic_id'] == topic_id]
                if not topic_name_row.empty:
                    custom_name = topic_name_row['CustomName'].iloc[0]
                    # Check if the custom name is valid and not the default
                    if pd.notna(custom_name) and not custom_name.lower().startswith("topic"):
                        labels.append(custom_name)
                        continue # Move to next topic if we found a good custom name

            # Fallback to using keywords if no valid CustomName exists
            topic_words = topic_model.get_topic(topic_id)
            if topic_words:
                # Take the first 3 keywords and join them for a concise label
                label = ", ".join([word for word, prob in topic_words[:3]])
                labels.append(label)

        return "\n".join(labels) if labels else ""

    except Exception as e:
        logging.error(f"Error getting shared topic for {channel_id_A}-{channel_id_B}: {e}")
        return "" # Return empty string on any error

print("Helper function 'get_shared_topic_context' is now defined.")

In [ ]:
# %%
# Cell: Visualization Function for Interactive Widget (Corrected)

def visualize_channel_neighborhood(selected_channel_id, degrees):
    """
    Queries data and draws the network neighborhood for the selected channel
    up to a specified degree of separation.
    """
    global currently_visualized_channel_id # Use the global-like variable
    with plot_output:
        plot_output.clear_output(wait=True);
        with message_output: message_output.clear_output()
        if not selected_channel_id: return

        selected_channel_row = channels_df[channels_df['id'] == selected_channel_id]
        if selected_channel_row.empty:
            with message_output: print(f"Selected channel ID {selected_channel_id} not found in loaded channel data.")
            return
        selected_channel_name_for_title = selected_channel_row['display_name'].iloc[0]

        print(f"Generating {degrees}-degree network for: {selected_channel_name_for_title}...")

        # Build a full graph object from the complete collaboration data to traverse it
        global G_full_for_viz
        if 'G_full_for_viz' not in globals() or not isinstance(G_full_for_viz, nx.Graph):
            print(" -> Creating full collaboration graph for traversal (one-time)...")
            G_full_for_viz = nx.from_pandas_edgelist(
                collab_df_full, 'channel_id_1', 'channel_id_2',
                edge_attr=['collaboration_count']
            )

        if not G_full_for_viz.has_node(selected_channel_id):
            with message_output: print(f"No collaboration data found for channel: {selected_channel_name_for_title}")
            return

        # Create the subgraph using the specified radius (degrees)
        subgraph = nx.ego_graph(G_full_for_viz, selected_channel_id, radius=degrees)

        # Performance Warning for large subgraphs
        if subgraph.number_of_nodes() > config.NETWORK_VIZ_MAX_SUBGRAPH_NODES:
            with message_output:
                display(HTML(f"<p style='color: orange;'><b>Warning:</b> The {degrees}-degree neighborhood has {subgraph.number_of_nodes()} nodes, which is more than the display limit of {config.NETWORK_VIZ_MAX_SUBGRAPH_NODES}. The graph may be slow and hard to read.</p>"))

        # --- Visualization logic ---
        # Get data for nodes in the subgraph
        node_data_viz = channels_df[channels_df['id'].isin(list(subgraph.nodes()))].set_index('id')

        missing_nodes = [n for n in list(subgraph.nodes()) if n not in node_data_viz.index]
        if missing_nodes:
            print(f"Warning: Missing channel data for nodes: {missing_nodes}")
            dummy_data = pd.DataFrame({'follower_count': 1, 'display_name': 'UNKNOWN'}, index=missing_nodes)
            node_data_viz = pd.concat([node_data_viz, dummy_data])

        # Calculate visual properties using the correct `subgraph` variable
        node_sizes_viz = [math.log10(max(1, node_data_viz.loc[node].get('follower_count', 1)) + 1) * 200 + 150 for node in subgraph.nodes()]
        node_labels_viz = {node: node_data_viz.loc[node, 'display_name'] for node in subgraph.nodes()}
        edge_widths_viz = [math.log10(max(0, d.get('weight', 0)) + 1) * 1.5 + 0.5 for u, v, d in subgraph.edges(data=True)]

        edge_labels_viz = {}
        print("Deriving edge context from BERTopic model...")
        for u, v in subgraph.edges():
            shared_topic_label = get_shared_topic_context(
                u, v,
                videos_with_topics=videos_with_topics_df,
                topic_model=topic_model
            )
            edge_labels_viz[(u, v)] = shared_topic_label

        # --- Plotting ---
        plt.figure(figsize=(16, 12));
        pos_viz = nx.spring_layout(subgraph, k=0.5, iterations=50, seed=42)

        # Use `subgraph` for all drawing functions
        nx.draw_networkx_nodes(subgraph, pos_viz, node_size=node_sizes_viz, node_color='skyblue', alpha=0.8)
        nx.draw_networkx_edges(subgraph, pos_viz, width=edge_widths_viz, alpha=0.4, edge_color='gray')
        nx.draw_networkx_labels(subgraph, pos_viz, labels=node_labels_viz, font_size=9)
        nx.draw_networkx_edge_labels(subgraph, pos_viz, edge_labels=edge_labels_viz, font_size=7, font_color='darkgreen',
                                     bbox=dict(facecolor='white', alpha=0.4, edgecolor='none', boxstyle='round,pad=0.2'))

        plt.title(f"{degrees}-Degree Collaboration Network for: {selected_channel_name_for_title}", fontsize=16);
        plt.axis('off');
        plt.tight_layout();
        plt.show()

        # When visualization is successful, store the channel ID
        currently_visualized_channel_id = selected_channel_id
        print("Done.")

In [ ]:
# %%
# Cell: Search and Selection Logic (Updated for Degrees of Separation)

def handle_search_click(b):
    """Function called when the search button is clicked."""
    global currently_visualized_channel_id
    search_term = search_input.value.strip()
    # Reset state
    results_dropdown.options = [("---", None)];
    results_dropdown.value = None;
    results_dropdown.disabled = True
    with message_output: message_output.clear_output()
    with plot_output: plot_output.clear_output()
    currently_visualized_channel_id = None

    if len(search_term) < 3:
        with message_output: print("Please enter at least 3 characters to search.")
        return

    print(f"Searching for channels matching: '{search_term}'...")
    search_pattern = f"%{search_term.lower()}%"
    cursor = db_conn.cursor()
    # Updated SQL to sort by follower count, showing more relevant results first
    sql = "SELECT id, login, display_name FROM Channels WHERE LOWER(login) LIKE ? OR LOWER(display_name) LIKE ? ORDER BY follower_count DESC LIMIT 50"
    try:
        cursor.execute(sql, (search_pattern, search_pattern));
        matches = cursor.fetchall()
    except sqlite3.Error as e:
        with message_output: print(f"Database search error: {e}"); return

    if not matches:
        with message_output: print(f"No channels found matching '{search_term}'.")
    elif len(matches) == 1:
        match = matches[0];
        channel_id = match['id']
        with message_output: print(f"Found 1 match: {match['display_name']} ({match['login']}). Visualizing...")
        visualize_channel_neighborhood(channel_id, degree_selector.value)
    else:
        match_options = [("--- Select a Match ---", None)] + [(f"{row['display_name']} ({row['login']})", row['id']) for row in matches]
        results_dropdown.options = match_options;
        results_dropdown.disabled = False
        with message_output: print(f"Found {len(matches)} matches. Please select one from the dropdown below.")


def handle_match_selection(change):
    """Function called when a channel is selected from the results dropdown."""
    selected_id = change.get('new')
    if selected_id:
        visualize_channel_neighborhood(selected_id, degree_selector.value)
    else:
        with plot_output: plot_output.clear_output()

# --- NEW: Handler for changing the degree of separation ---
def handle_degree_change(change):
    """Function called when the degree selector dropdown changes."""
    new_degree = change.get('new')
    # If a channel is already selected/visualized, re-run the visualization with the new degree
    if currently_visualized_channel_id:
        visualize_channel_neighborhood(currently_visualized_channel_id, new_degree)

# Link events to handlers
search_button.on_click(handle_search_click)
results_dropdown.observe(handle_match_selection, names='value')
degree_selector.observe(handle_degree_change, names='value')

In [ ]:
# %%
# Cell: Display Interactive Widgets
print("\n--- Displaying Interactive Network Visualization Widget ---")

# Arrange search input and button horizontally
search_box = HBox([search_input, search_button])
# Arrange degree and results dropdowns horizontally
selector_box = HBox([degree_selector, results_dropdown])

# Arrange all controls vertically for a clean layout
controls = VBox([
    search_box,
    selector_box,
    message_output,
    plot_output
    ], layout=Layout(width='100%'))

display(controls)

In [ ]:
# Cell 24: Final Closing Remarks & Cleanup
print(f"\n--- Notebook Processing Finished at {datetime.now().strftime('%H:%M:%S')} ---")
print(
    "You can re-run cells like 'Run Data Collection Cycle', 'Mention Processing Loop', or 'Run Channel Refresh Cycle' to gather more data.")
print("Consider closing the database connection manually if you are completely finished.")
# Example manual close (uncomment to run):
# if 'db_conn' in locals() and db_conn is not None:
#     try:
#         db_conn.close()
#         print("Database connection closed.")
#         db_conn = None # Clear variable
#     except Exception as e:
#         print(f"Error closing database connection: {e}")